# ResNet50

Instalando requisitos

In [1]:
%pip install -r ./app/requirements.txt -q
%pip install pandas -q
%pip install scikit-learn -q
%pip install tensorflow -q
%pip install numpy -q

You should consider upgrading via the '/mnt/home/henrique/Repositorios_Git/GitHub.com/Henriquemcc/Ciencia da Computacao/Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1/develop_henrique_2024_05_27/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/mnt/home/henrique/Repositorios_Git/GitHub.com/Henriquemcc/Ciencia da Computacao/Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1/develop_henrique_2024_05_27/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/mnt/home/henrique/Repositorios_Git/GitHub.com/Henriquemcc/Ciencia da Computacao/Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1/develop_henrique_2024_05_27/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You 

Importando requisitos

In [2]:
from app.modelo.imagem_rgb import ImagemRGB
import sklearn.model_selection
import pandas
import tensorflow
import os
import numpy
import pickle
import random

2024-05-30 13:39:38.225808: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 13:39:39.399652: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Definindo variáveis

In [3]:
image_directory = os.path.join(os.getcwd(), 'dataset_converted')
num_classes = 6

Abrindo base de dados

In [4]:
data_frame = pandas.read_csv("classifications.csv")

Adicionando coluna path

In [5]:
for index, row in data_frame.iterrows():
    data_frame.at[index, 'path'] = os.path.join(image_directory, row['bethesda_system'], "{}.png".format(row['cell_id']))

In [6]:
data_frame

,image_id,image_filename,image_doi,cell_id,bethesda_system,nucleus_x,nucleus_y,path
0,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,1,SCC,792,462,/mnt/home/henrique/Repositorios_Git/GitHub.com...
1,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,2,SCC,601,678,/mnt/home/henrique/Repositorios_Git/GitHub.com...
2,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,3,SCC,363,467,/mnt/home/henrique/Repositorios_Git/GitHub.com...
3,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,4,SCC,599,437,/mnt/home/henrique/Repositorios_Git/GitHub.com...
4,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,5,Negative for intraepithelial lesion,1186,450,/mnt/home/henrique/Repositorios_Git/GitHub.com...
...,...,...,...,...,...,...,...,...
11529,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11530,LSIL,618,407,/mnt/home/henrique/Repositorios_Git/GitHub.com...
11530,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11531,LSIL,607,374,/mnt/home/henrique/Repositorios_Git/GitHub.com...
11531,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11532,LSIL,707,251,/mnt/home/henrique/Repositorios_Git/GitHub.com...
11532,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11533,LSIL,579,246,/mnt/home/henrique/Repositorios_Git/GitHub.com...


Adicionando coluna random e ordenando por meio dela

In [7]:
data_frame['random'] = data_frame[data_frame.columns[0]].apply(lambda _: random.random())

In [8]:
data_frame.sort_values(
    by='random',
    inplace=True
)

Removendo imagens que não existem

In [9]:
for index, row in data_frame.iterrows():
    if not os.path.exists(row['path']):
        data_frame.drop(index, inplace=True)

In [10]:
data_frame

,image_id,image_filename,image_doi,cell_id,bethesda_system,nucleus_x,nucleus_y,path,random
989,379,326be63a57e19af9bff9a7df786b9987.png,10.6084/m9.figshare.12230819,990,ASC-US,558,555,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.000007
11485,2,9d0f43b29d78f673e9caf77928b4af3c.png,10.6084/m9.figshare.12229520,11486,ASC-US,739,254,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.000324
4164,242,36ec7e0332c9ab3aa1edb53fe8f6cba8.png,10.6084/m9.figshare.12230372,4165,Negative for intraepithelial lesion,349,79,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.000478
5113,194,040722a73af21747a3b66a8d69a235d0.png,10.6084/m9.figshare.12230201,5114,Negative for intraepithelial lesion,1197,540,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.000906
7577,103,b0fddd5dbd774fcc23ddf897ee7bf26c.png,10.6084/m9.figshare.12229874,7578,Negative for intraepithelial lesion,860,974,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.001269
...,...,...,...,...,...,...,...,...,...
6410,138,c0d57801655afd0ab48ad3f1e4b43603.png,10.6084/m9.figshare.12230015,6411,Negative for intraepithelial lesion,753,656,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.999122
4262,238,3f30a996dcac3c4845ba2e650fe874b5.png,10.6084/m9.figshare.12230360,4263,LSIL,619,827,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.999213
4250,239,f5cad8f1a038b616cb2ce4ca46183262.png,10.6084/m9.figshare.12230363,4251,Negative for intraepithelial lesion,671,498,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.999291
10193,39,dd569f426f91cc5d6a20804c800d6eb3.png,10.6084/m9.figshare.12229643,10194,LSIL,719,326,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.999690


Separando o x

In [11]:
x = []
for index, row in data_frame.iterrows():
    x.append(row['path'])
x = pandas.array(x)

In [12]:
x

<StringArray>
[                               '/mnt/home/henrique/Repositorios_Git/GitHub.com/Henriquemcc/Ciencia da Computacao/Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1/develop_henrique_2024_05_27/dataset_converted/ASC-US/990.png',
                              '/mnt/home/henrique/Repositorios_Git/GitHub.com/Henriquemcc/Ciencia da Computacao/Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1/develop_henrique_2024_05_27/dataset_converted/ASC-US/11486.png',
  '/mnt/home/henrique/Repositorios_Git/GitHub.com/Henriquemcc/Ciencia da Computacao/Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1/develop_henrique_2024_05_27/dataset_converted/Negative for intraepithelial lesion/4165.png',
  '/mnt/home/henrique/Repositorios_Git/GitHub.com/Henriquemcc/Ciencia da Computacao/Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1/develop_henrique_2024_05_27/dataset_converted/Negative for intraepithelial lesion/5114.png',
  '/mnt/home/henrique/Repo

In [13]:
x.shape

(5581,)

Separando o y

In [14]:
y = data_frame.iloc[:, 4].values
label_encoder = sklearn.preprocessing.LabelEncoder()
y = label_encoder.fit_transform(y)

In [15]:
type(y)

numpy.ndarray

In [16]:
y.shape

(5581,)

Separando o conjunto de treino e teste

In [17]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y)

In [18]:
x_train.shape

(4185,)

In [19]:
y_train.shape

(4185,)

In [20]:
x_test.shape

(1396,)

In [21]:
y_test.shape

(1396,)

Exportando x_test, x_train, y_test, y_train para um arquivo externo

In [22]:
with open('data.pkl', mode='wb') as file:
    pickle.dump([x_train, x_test, y_train, y_test], file)

# Aplicando o ResNet50

Construindo o modelo

In [23]:
input_tensor = tensorflow.keras.layers.Input(shape=(100, 100, 3))
modelo_base = tensorflow.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)
mbo = modelo_base.output
mbo = tensorflow.keras.layers.GlobalAveragePooling2D()(mbo)
mbo = tensorflow.keras.layers.Dense(1024, activation='relu')(mbo)
predicoes = tensorflow.keras.layers.Dense(num_classes, activation='softmax')(mbo)

modelo = tensorflow.keras.models.Model(inputs=modelo_base.input, outputs=predicoes)

for layer in modelo_base.layers:
    layer.trainable = False

2024-05-30 13:39:50.257301: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-30 13:39:50.437814: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Compilando

In [24]:
modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'recall', 'precision'])

Treinando o modelo

In [25]:
def preprocessar_imagem(imagem):
    if isinstance(imagem, numpy.ndarray):
        imagem = imagem/255.0
    else:
        raise ValueError("Imagem deve ser um numpy array")
    
    imagem = tensorflow.convert_to_tensor(imagem, dtype=tensorflow.float32)
    imagem = tensorflow.expand_dims(imagem, axis=0)
    return imagem

In [26]:
for index, image_path in enumerate(x_train):
    if (os.path.exists(image_path)):
        imagem_rgb = ImagemRGB.from_file(image_path)
        imagem_preprocessada = preprocessar_imagem(imagem_rgb.matriz)
        label = tensorflow.keras.utils.to_categorical(y_train[index], num_classes)
        label = tensorflow.convert_to_tensor(label, dtype=tensorflow.int8)
        label = tensorflow.expand_dims(label, axis=0)
        modelo.fit(x=numpy.array(imagem_preprocessada), y=label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.0000e+00 - loss: 2.3214 - precision: 0.0000e+00 - recall: 0.0000e+00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 1.0000 - loss: 0.0060 - precision: 1.0000 - recall: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.0000e+00 - loss: 10.3449 - precision: 0.0000e+00 - recall: 0.0000e+00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 1.0000 - loss: 3.0513e-04 - precision: 1.0000 - recall: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 1.0000 - loss: 0.0021 - precision: 1.0000 - recall: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 1.0000 - loss: 0.0086 - precision: 1.0000 - recall: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 1.0000 - loss: 0.0359 - precision: 1.0000 - recall: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 1.0000 - loss: 0.0935 - precision: 1.0000 - recall: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 1.0000 - loss: 0.1492 - precision: 1.0000 - recall

KeyboardInterrupt: 

Exportando o modelo treinado

In [ ]:
modelo.save('modelo.keras')

Exportando os pesos

In [ ]:
modelo.save_weights('pesos.weights.h5')

Testando o modelo

In [ ]:
predicoes = []
for index, image_path in enumerate(x_test):
    if (os.path.exists(image_path)):
        imagem_rgb = ImagemRGB.from_file(image_path)
        imagem_preprocessada = preprocessar_imagem(imagem_rgb.matriz)
        predicao = modelo.predict(imagem_preprocessada)
        predicoes.append(predicao)
predicoes = numpy.array(predicoes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━

Exportando as predições

In [ ]:
with open('predicoes.pkl', mode='wb') as file:
    pickle.dump(predicoes, file)

In [ ]:
predicoes

array([[[0.02351419, 0.08161768, 0.08058541, 0.10495577, 0.6956503 ,
         0.01367676]],

       [[0.0210823 , 0.0771265 , 0.0759612 , 0.09986185, 0.7139601 ,
         0.01200797]],

       [[0.01867094, 0.07226446, 0.07078291, 0.09453884, 0.7332476 ,
         0.01049528]],

       ...,

       [[0.0176205 , 0.06964742, 0.06767683, 0.09263962, 0.74242705,
         0.00998859]],

       [[0.01694301, 0.06823891, 0.06617898, 0.09090581, 0.74814814,
         0.00958524]],

       [[0.02004587, 0.07478126, 0.07307739, 0.09809575, 0.7224454 ,
         0.01155424]]], dtype=float32)

In [ ]:
import sklearn.metrics

In [ ]:
predicoes.shape

(1416, 1, 6)

In [ ]:
y_test.shape

(2884,)

In [ ]:
sklearn.metrics.classification_report(y_teste, predicoes)

NameError: name 'y_teste' is not defined